# Introductory Tutorial

## Instalation

If the SOBA package is not yet installed, we must first do so. To install SOBA the best option is to use the package management system PIP. For this, we execute the following command.

```bash
    $ pip install soba
```

In case of error, this other command should be used, ensuring to have installed python 3 and pip 3.

```bash
    $ pip3 install soba
```

## Tutorial

The SOBA tool can be provided to be used directly on two scenarios:

1. Generic case with a space defined as a grid of a given square size (by default, half a meter on each side).
2. Simplified case with a room defined by rooms, to perform simulations in simplified buildings that require less consumption of resources and specifications.

An introductory tutorial will be presented for each case, although most parameters are common or similar.

SOBA enables the performance of the simulations in two modes:

1. With visual representation.
2. In batch mode. 

In the tutorials, the small modifications required to use each posibility are reflected.


In addition, two added mechanisms are provided to interact with the simulation:

1. Use an API on a REST server to obtain information and create and manage avatars.
2. use the external tool [RAMEN](https://github.com/gsi-upm/RAMEN) for advanced 3D-visualization on Three.js. 

IMPORTANT NOTE: The .py files described in this tutorial are available in the github repository 
https://github.com/gsi-upm/soba/tree/master/projects/basicExamples

### Implementing  a sample model with continuous space

Once soba is installed, the implementation can be started. First we define the generic parameters to both types of scenario.

1.- We define the characteristics of the occupants

In [9]:
from collections import OrderedDict
#JSON to store all the informacion.
jsonsOccupants = []

#Number of occupants
N = 3

#Definition of the states
states = OrderedDict([('Leaving','out'), ('Resting', 'sofa'), ('Working in my laboratory', 'wp')])

#Definition of the schedule
schedule = {'t1': "09:00:00", 't2': "13:00:00", 't3': "14:10:00"}

#Possible Variation on the schedule
variation = {'t1': "00:10:00", 't2': "01:20:00", 't3': "00:20:00"}

#Probability of state change associated with the Markovian chain as a function of the temporal period
markovActivity = {
    '-t1': [[100, 0, 0], [0, 0, 0], [0, 0, 0]],
    't1-t2': [[0, 0, 100], [0, 50, 50], [0, 50, 50]],
    't2-t3': [[100, 0, 0], [0, 50, 50], [0, 50, 50]],
    't3-': [[0, 0, 100], [0, 100, 0], [0, 100, 0]]
}

#Time associated to each state (minutes)
timeActivity = {
    '-t1': [1, 0, 0], 't1-t2': [10, 10, 10], 't2-t3': [10, 10, 10], 't3-': [1, 1, 1]
}

#Time variation associated to each state (minutes)
timeActivityVariation = {
    '-t1': [0, 0, 0], 't1-t2': [0, 0, 0], 't2-t3': [0, 0, 0], 't3-': [0, 0, 0]
}

#Store the information
jsonOccupant = {'type': 'example' , 'N': N, 'states': states , 'schedule': schedule, 'variation': variation,
'markovActivity': markovActivity, 'timeActivity': timeActivity, "timeActivityVariation": timeActivityVariation}

jsonsOccupants.append(jsonOccupant)

2.- We define the building plan or the distribution of the space.

In [10]:
import soba.visualization.ramen.mapGenerator as ramen

with open('labgsi.blueprint3d') as data_file:
    jsonMap = ramen.returnMap(data_file)

3.- We implement a Model inheriting a base class of SOBA.

In [15]:
from soba.models.roomsModel import RoomsModel
from time import time

class ModelExample(RoomsModel):

    def __init__(self, width, height, jsonMap, jsonsOccupants, seed = int(time())):
        super().__init__(width, height, jsonMap, jsonsOccupants, seed = seed, timeByStep = 0.1)
        self.createOccupants(jsonsOccupants)

    def step(self):
        if self.clock.clock.hour > 17:
            self.finishSimulation = True
        super().step()

4.- We call the execution methods.

    4.1-With visual representation.

In [16]:
import soba.run
import os.system

parameters = {'width': 40, 'height': 40, 'jsonMap': jsonMap, 'jsonsOccupants': jsonsOccupants}

os.system(soba.run.run(ModelExample, parameters, visualJS="example.js"))


 Wrong params :(

   Options:
	-v,		Visual option on browser
	-b,		Background option
	-r,		Ramen option
 


    4.1- Bacth mode.

In [ ]:
import soba.run

#Fixed parameters during iterations
fixed_params = {"width": 40, "height": 40, "jsonMap": jsonMap, "jsonsOccupants": jsonsOccupants}
#Variable parameters to each iteration
variable_params = {"seed": range(10, 500, 10)}

soba.run.run(ModelExample, fixed_params, variable_params)

### Implementing  a sample model with simplified space

Once soba is installed, the implementation can be started. First we define the generic parameters to both types of scenario.

1.- We define the characteristics of the occupants


In [4]:
from collections import OrderedDict
#JSON to store all the informacion.
jsonsOccupants = []

#Number of occupants
N = 3

#Definition of the states
states = OrderedDict([('out','Pos1'), ('Working in my laboratory', {'Pos2': 1, 'Pos3': 2})])

#Definition of the schedule
schedule = {'t1': "09:00:00", 't2': "13:00:00", 't3': "14:10:00"}

#Possible Variation on the schedule
variation = {'t1': "00:10:00", 't2': "01:20:00", 't3': "00:20:00"}

#Probability of state change associated with the Markovian chain as a function of the temporal period.
markovActivity = {
    '-t1': [[100, 0], [0, 0]],
    't1-t2': [[50, 50], [0, 0]],
    't2-t3': [[0, 0], [50, 0]],
    't3-': [[0, 50], [10, 90]]
}

#Time associated to each state (minutes)
timeActivity = {
    '-t1': [60, 0],
    't1-t2': [2, 60],
    't2-t3': [60, 10],
    't3-': [60, 20]
}

#Store the information
jsonOccupant = {'type': 'example' , 'N': N, 'states': states , 'schedule': schedule, 'variation': variation, 
                'markovActivity': markovActivity, 'timeActivity': timeActivity}
jsonsOccupants.append(jsonOccupant)

2.- We define the building plan or the distribution of the space.

In [5]:
jsonMap = {
  'Pos1': {'entrance':'', 'conectedTo': {'U':'Pos2'}, 'measures': {'dx':2, 'dy':2}},
  'Pos2': {'measures': {'dx':3, 'dy':3.5}, 'conectedTo': {'R':'Pos3'}},
  'Pos3': {'measures': {'dx':3, 'dy':3.5}}
}

3.- We implement a Model inheriting a base class of SOBA.

In [6]:
from soba.model.model import ContinuousModel
import datetime as dt

class ModelExample(RoomsModel):

    def __init__(self, width, height, jsonMap, jsonsOccupants, seed = dt.datetime.now()):
        super().__init__(width, height, jsonMap, jsonsOccupants, seed = seed)

    def step(self):
        if self.clock.clock.day > 3:
            self.finishSimulation = True
        super().step()

ImportError: No module named 'soba.model'

4.- We call the execution methods.
    4.1- With visual representation.

In [7]:
cellW = 4
cellH = 4

soba.run.run(ModelExample, [], cellW, cellH, jsonMap, jsonsOccupants)

NameError: name 'soba' is not defined

    4.1- Bacth mode.

In [ ]:
#Fixed parameters during iterations
fixed_params = {"width": cellW, "height": cellH, "jsonMap": jsonMap, "jsonsOccupants": jsonsOccupants}
#Variable parameters to each iteration
variable_params = {"seed": range(10, 500, 10)}

soba.run.run(ModelExample, fixed_params, variable_params)

## Running the simulation using the terminal

```bash
    $ python exampleContinuous.py -v
```

Options:

	-v,		Visual option on browser

	-b,		Background option
    
	-r,		Ramen option
